In [1]:
import pandas as pd
import numpy as np
import utils

READ_PARQUET = True

In [2]:
if READ_PARQUET:
    print("Reading parquet.")
    df = utils.load_parquet()
else:
    print("Reading excel and savings as parquet.")
    df = utils.load_excel()
    utils.save_parquet(df)

Reading parquet.


In [3]:
from copy import deepcopy
# drop uncomplete cases (2017 mostly) and thus do not have a complete label

cases_df = deepcopy(df.loc[df['Complete Timestamp'].dt.year < 2017])

## Undesired Outcome 1

Undesired outcome 1: The payment is late. A payment can be considered
timely, if there has been a begin payment activity by the end of the year that was
not eventually followed by abort payment.

Late Payments There is no easy way of filtering the cases to find cases with
late payments. So in order to look at the differences, we added a case attribute
ourselves with the use of Python. With the use of the ProM forum, we identified
3 situations in which a case is considered late:
1. There is no ‘begin payment’ event in the case.
2. The last ‘begin payment’ event is followed by an ‘abort payment’ event.
3. The last ‘begin payment’ event occurs in a later year than what the case was
started in.

### 1. There is no 'begin payment' event in the case 

#### First determine the latest Begin Payment activity for every case
__TO DO__
- check if latest begin payment idxmax goes well for cases with only 1 Begin Payment event


In [4]:
# group by per caseid --> to find latest payment

df = cases_df.loc[cases_df['Activity'] == 'Payment application-Application-begin payment']
df = df.rename(columns = {'Complete Timestamp': 'Timestamp'})
latest_payments = df.loc[df.groupby('Case ID').Timestamp.idxmax()]


for i in latest_payments.index:
    # Add column to inital dataframe
    cases_df.loc[i,'_Latest_Begin_Payment'] = 1


#### Determine all cases without Begin Payment event

In [5]:
# find all case id's for which the sum of Latest begin payment = 0. --> indicates cases without Begin Payment event
payments = cases_df[['Case ID', '_Latest_Begin_Payment']].groupby(by=['Case ID']).sum()

no_begin_payment = payments.loc[payments['_Latest_Begin_Payment'] == 0]

# in current subset of data there is no sensor without begin payment
no_begin_payment

,_Latest_Begin_Payment
Case ID,
06501822341c30a8,0.0
25ed5ff468922bab,0.0


#### Label all cases without Begin Payment event as Undesired Outcome 1

In [6]:

for i in no_begin_payment.index:
    # for all cases without begin payment --> label them as undesired outcome rows.
    cases_df.loc[cases_df['Case ID'] == i, 'UndesiredOutcome1'] = 1
    

In [7]:
cases_df.loc[cases_df['UndesiredOutcome1'] == 1].head()

,Case ID,Activity,Resource,Complete Timestamp,Variant,Variant index,(case) amount_applied0,(case) amount_applied1,(case) amount_applied2,(case) amount_applied3,...,concept:name,docid,doctype,eventid,lifecycle:transition,note,subprocess,success,_Latest_Begin_Payment,UndesiredOutcome1
95017,25ed5ff468922bab,Payment application-Application-mail income,0;n/a,2015-05-15,Variant 7472,7472,7333.46,NaN,NaN,NaN,...,mail income,-90066104691272912,Payment application,NaN,complete,none,Application,True,NaN,1.0
95018,25ed5ff468922bab,Payment application-Application-mail valid,0;n/a,2015-05-15,Variant 7472,7472,7333.46,NaN,NaN,NaN,...,mail valid,-90066104691272912,Payment application,NaN,complete,none,Application,True,NaN,1.0
95019,25ed5ff468922bab,Entitlement application-Main-mail valid,0;n/a,2015-05-15,Variant 7472,7472,7333.46,NaN,NaN,NaN,...,mail valid,-108080539086341280,Entitlement application,NaN,complete,none,Main,True,NaN,1.0
95020,25ed5ff468922bab,Entitlement application-Main-mail valid,0;n/a,2015-05-15,Variant 7472,7472,7333.46,NaN,NaN,NaN,...,mail valid,-108080539086341280,Entitlement application,NaN,complete,none,Main,True,NaN,1.0
95021,25ed5ff468922bab,Parcel document-Main-initialize,019209,2016-05-12,Variant 7472,7472,7333.46,NaN,NaN,NaN,...,initialize,-108080539086343168,Parcel document,-1.080805e+17,complete,none,Main,True,NaN,1.0


### 2. The last ‘begin payment’ event is followed by an ‘abort payment’ event. 

#### Determine all 'latest abort payment' events

In [8]:
# Add column that indicates the latest abort payment 

abort_df = cases_df.loc[cases_df['Activity'] == 'Payment application-Application-abort payment']
abort_df = abort_df.rename(columns = {'Complete Timestamp': 'Timestamp'})
latest_aborts = abort_df.loc[abort_df.groupby('Case ID').Timestamp.idxmax()]

for i in latest_aborts.index:
    
    cases_df.loc[i,'_Latest_Abort_Payment'] = 1

In [9]:
# insert artificial row that complies with the criteria

cases_df.loc[cases_df['Case ID'] == '5A']

,Case ID,Activity,Resource,Complete Timestamp,Variant,Variant index,(case) amount_applied0,(case) amount_applied1,(case) amount_applied2,(case) amount_applied3,...,docid,doctype,eventid,lifecycle:transition,note,subprocess,success,_Latest_Begin_Payment,UndesiredOutcome1,_Latest_Abort_Payment
0,5A,Payment application-Application-begin payment,0;n/a,2015-04-16,Variant 6000,6000,401.63,NaN,NaN,NaN,...,-18008611495635268,Payment application,NaN,complete,none,Application,True,1.0,NaN,NaN
1,5A,Payment application-Application-abort payment,0;n/a,2015-04-18,Variant 6000,6000,401.63,NaN,NaN,NaN,...,-18008611495635268,Payment application,NaN,complete,none,Application,True,NaN,NaN,1.0


In [10]:
# filter on criteria: cases with a begin payment and an abort payment
latest_df = cases_df.loc[(cases_df['_Latest_Begin_Payment'] == 1) | (cases_df['_Latest_Abort_Payment'] ==1)]

#### Find cases with abort after begin payment 
- What to do when abort payment and begin payment have the same timestamp? --> timestamp is actually a date rather than a timestamp
- Did we lose timestamp info when exporting from disco or rapidminer??
- In code a mistake
- Do things differently for automatic (batch) processing operations??

In [11]:
# filter cases with date of abort payment activity > date of latest_begin_payment

# Each case id has at max 2 rows. This because we only consider the LATEST begin payment and LATEST abort payment

grouped_df = latest_df.groupby('Case ID')['Complete Timestamp'].agg(['min','max']).rename(columns={'min':'first','max':'last'})

merged_df = pd.merge(latest_df, grouped_df, left_on='Case ID', right_on='Case ID')


# check if timestamp of latest abort payment row  == max --> then there is an abort payment after the latest begin payment
for i in merged_df.index:
    
    row = merged_df.loc[i, :]
    
    if row['_Latest_Abort_Payment'] == 1 and row['Complete Timestamp'] == row['last'] and row['first'] != row['last']: # check timestamps
        # when first and last date are exactly the same, the order cannot be concluded from the data and hence these cases cannot be used for predictions
        merged_df.loc[i, '_Abort_After_Begin_Payment'] = 1

        
# label case as undesired outcome 1 in 'data' dataframe
try:
    aborted_cases =  list(merged_df['Case ID'].loc[merged_df['_Abort_After_Begin_Payment'] == 1].unique())
    for i in aborted_cases:
            cases_df.loc[cases_df['Case ID'] == i,'_Abort_After_Begin_Payment'] = 1

            cases_df.loc[cases_df['Case ID'] == i,'UndesiredOutcome1'] = 1
except:
    print("There is no such case with Abort Payment event after Last Begin Payment event")



In [12]:
# check if artificially created rows are indeed labeled as abort_after_begin_payment = 1
merged_df.loc[merged_df['_Abort_After_Begin_Payment'] == 1]


,Case ID,Activity,Resource,Complete Timestamp,Variant,Variant index,(case) amount_applied0,(case) amount_applied1,(case) amount_applied2,(case) amount_applied3,...,lifecycle:transition,note,subprocess,success,_Latest_Begin_Payment,UndesiredOutcome1,_Latest_Abort_Payment,first,last,_Abort_After_Begin_Payment
1,5A,Payment application-Application-abort payment,0;n/a,2015-04-18,Variant 6000,6000,401.63,NaN,NaN,NaN,...,complete,none,Application,True,NaN,NaN,1.0,2015-04-16,2015-04-18,1.0


#### Conclusion
- Timestamps are not accurate as they only indicate the date and not the timestamp.
- A lot of cases have 'Complete Timestamp' value the same for latest 'Begin Payment' as latest 'Abort Payment'! --> seems to be nothing we can do about this.

### 3. The last ‘begin payment’ event occurs in a later year than what the case was started in.

In [13]:
# determine case start year
start_year_case = cases_df.groupby(by=['Case ID'])['Complete Timestamp'].agg(['min']).rename(columns={'min':'StartDate'})
start_year_case['StartYear'] = start_year_case['StartDate'].dt.year
start_year_case = start_year_case.reset_index()
start_year_case.head()


,Case ID,StartDate,StartYear
0,000612b48d30de74,2015-05-06,2015
1,0032b59241ce3589,2015-04-30,2015
2,0042f6f2a63d2cb6,2015-04-27,2015
3,004d4aadd6785efd,2015-05-06,2015
4,005e4fdaf08da672,2015-05-07,2015


In [14]:
# determine latest begin payment year
latest_begin_payment_year = cases_df.loc[cases_df['Activity'] == 'Payment application-Application-abort payment'].groupby(by=['Case ID'])['Complete Timestamp'].agg(['min']).rename(columns={'min':'StartDate'})
latest_begin_payment_year['_Latest_Begin_Payment_Year'] =  latest_begin_payment_year['StartDate'].dt.year
latest_begin_payment_year = latest_begin_payment_year.reset_index()
latest_begin_payment_year.head()

,Case ID,StartDate,_Latest_Begin_Payment_Year
0,000612b48d30de74,2015-12-18,2015
1,0032b59241ce3589,2015-12-18,2015
2,0042f6f2a63d2cb6,2015-12-16,2015
3,005e4fdaf08da672,2015-12-18,2015
4,0063e4bb2015f0ba,2015-12-18,2015


In [15]:
# merge two dataframes
year_df = pd.merge(latest_begin_payment_year, start_year_case, on='Case ID')
del year_df['StartDate_x'] # remove
del year_df[ 'StartDate_y'] # remove
# Case ID is the unique identifier in merged_df and thus no grouping by is necessary
year_df['LatePayment'] = np.where(year_df['_Latest_Begin_Payment_Year'] > year_df['StartYear'], 1 , 0)

In [16]:
# cases with Latest Begin Payment event in a later year than the case has started
year_df.loc[year_df['LatePayment'] == 1]

,Case ID,_Latest_Begin_Payment_Year,StartYear,LatePayment
247,13404f41b918553d,2016,2015,1
455,260155273269fe3e,2016,2015,1
2373,cc845befea39d489,2016,2015,1
2405,cedcde09b55a5da7,2015,2014,1
2628,e19f821d86fc80eb,2016,2015,1
2769,ecbbb4234da32502,2016,2015,1


#### Label cases with payment in a later year than in which case started as undesired outcome 1

In [17]:

late_payments = year_df['Case ID'].loc[year_df['LatePayment'] == 1].unique()

# loop over all Case ID's that have a late payment
for i in late_payments:
    row = cases_df.loc[cases_df['Case ID'] == i, :]
    cases_df.loc[cases_df['Case ID'] == i,'_Begin_Payment_Next_Year'] = 1

    cases_df.loc[cases_df['Case ID'] == i,'UndesiredOutcome1'] = 1


In [18]:
#cases_df.loc[cases_df['Case ID'] == 'cc845befea39d489'].head()

In [19]:
cases_df.head()

,Case ID,Activity,Resource,Complete Timestamp,Variant,Variant index,(case) amount_applied0,(case) amount_applied1,(case) amount_applied2,(case) amount_applied3,...,eventid,lifecycle:transition,note,subprocess,success,_Latest_Begin_Payment,UndesiredOutcome1,_Latest_Abort_Payment,_Abort_After_Begin_Payment,_Begin_Payment_Next_Year
0,5A,Payment application-Application-begin payment,0;n/a,2015-04-16,Variant 6000,6000,401.63,NaN,NaN,NaN,...,NaN,complete,none,Application,True,1.0,1.0,NaN,1.0,NaN
1,5A,Payment application-Application-abort payment,0;n/a,2015-04-18,Variant 6000,6000,401.63,NaN,NaN,NaN,...,NaN,complete,none,Application,True,NaN,1.0,1.0,1.0,NaN
2,788e3f355afbed07,Payment application-Application-mail income,0;n/a,2015-04-16,Variant 6000,6000,401.63,NaN,NaN,NaN,...,NaN,complete,none,Application,True,NaN,NaN,NaN,NaN,NaN
3,788e3f355afbed07,Entitlement application-Main-mail valid,0;n/a,2015-04-27,Variant 6000,6000,401.63,NaN,NaN,NaN,...,NaN,complete,none,Main,True,NaN,NaN,NaN,NaN,NaN
4,788e3f355afbed07,Payment application-Application-mail valid,0;n/a,2015-04-27,Variant 6000,6000,401.63,NaN,NaN,NaN,...,NaN,complete,none,Application,True,NaN,NaN,NaN,NaN,NaN


### Save Undesired Outcomes as dataframe in parquet file

In [20]:
utils.save_parquet(cases_df, undesired_outcomes=True)

## Undesired Outcome 2

In [21]:
utils.generate_one_hot_encoding()

,Case ID,Activity,Sub Process,Application,Change,Main,Objection,On,Remote
0,5A,Payment application-Application-begin payment,Application,1.0,0.0,0.0,0.0,0.0,0.0
1,5A,Payment application-Application-abort payment,Application,1.0,0.0,0.0,0.0,0.0,0.0
2,788e3f355afbed07,Payment application-Application-mail income,Application,1.0,0.0,0.0,0.0,0.0,0.0
3,788e3f355afbed07,Entitlement application-Main-mail valid,Main,0.0,0.0,1.0,0.0,0.0,0.0
4,788e3f355afbed07,Payment application-Application-mail valid,Application,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
193677,4c1dce27d1fa14a5,Parcel document-Main-save,Main,0.0,0.0,1.0,0.0,0.0,0.0
193678,4c1dce27d1fa14a5,Parcel document-Main-check,Main,0.0,0.0,1.0,0.0,0.0,0.0
193679,4c1dce27d1fa14a5,Parcel document-Main-finish editing,Main,0.0,0.0,1.0,0.0,0.0,0.0
193680,4c1dce27d1fa14a5,Control summary-Main-save,Main,0.0,0.0,1.0,0.0,0.0,0.0


In [23]:
utils.generate_outcome_two()

,Case ID,Undesired Outcome 2
0,5A,False
1,5A,False
2,788e3f355afbed07,False
3,788e3f355afbed07,False
4,788e3f355afbed07,False
...,...,...
193677,4c1dce27d1fa14a5,False
193678,4c1dce27d1fa14a5,False
193679,4c1dce27d1fa14a5,False
193680,4c1dce27d1fa14a5,False


### Undesired Outcomes DataFrame